In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin
from tqdm import tqdm
from portiloopml.portiloop_python.ANN.wamsley_utils import detect_lacourse
import os

In [19]:


def shift_numpy(arr, num, fill_value=np.nan):
    result = np.empty_like(arr)
    if num > 0:
        result[:num] = fill_value
        result[num:] = arr[:-num]
    elif num < 0:
        result[num:] = fill_value
        result[:num] = arr[-num:]
    else:
        result[:] = arr
    return result

class FIR:
    def __init__(self, nb_channels, coefficients, buffer=None):
        
        self.coefficients = np.expand_dims(np.array(coefficients), axis=1)
        self.taps = len(self.coefficients)
        self.nb_channels = nb_channels
        self.buffer = np.array(buffer) if buffer is not None else np.zeros((self.taps, self.nb_channels))
    
    def filter(self, x):
        self.buffer = shift_numpy(self.buffer, 1, x)
        filtered = np.sum(self.buffer * self.coefficients, axis=0)
        return filtered

    
class FilterPipeline:
    def __init__(self,
                 nb_channels,
                 sampling_rate,
                 power_line_fq=60,
                 use_custom_fir=False,
                 custom_fir_order=20,
                 custom_fir_cutoff=30,
                 alpha_avg=0.1,
                 alpha_std=0.001,
                 epsilon=0.000001,
                 filter_args=[]):
        if len(filter_args) > 0:
            use_fir, use_notch, use_std = filter_args
        else:
            use_fir=True,
            use_notch=False,
            use_std=True
        self.use_fir = use_fir
        self.use_notch = use_notch
        self.use_std = use_std
        self.nb_channels = nb_channels
        assert power_line_fq in [50, 60], f"The only supported power line frequencies are 50 Hz and 60 Hz"
        if power_line_fq == 60:
            self.notch_coeff1 = -0.12478308884588535
            self.notch_coeff2 = 0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.12478308884588535
            self.notch_coeff5 = 0.99364593398236511
        else:
            self.notch_coeff1 = -0.61410695998423581
            self.notch_coeff2 =  0.98729186796473023
            self.notch_coeff3 = 0.99364593398236511
            self.notch_coeff4 = -0.61410695998423581
            self.notch_coeff5 = 0.99364593398236511
        self.dfs = [np.zeros(self.nb_channels), np.zeros(self.nb_channels)]
        
        self.moving_average = None
        self.moving_variance = np.zeros(self.nb_channels)
        self.ALPHA_AVG = alpha_avg
        self.ALPHA_STD = alpha_std
        self.EPSILON = epsilon
        
        if use_custom_fir:
            self.fir_coef = firwin(numtaps=custom_fir_order+1, cutoff=custom_fir_cutoff, fs=sampling_rate)
        else:
            self.fir_coef = [
                0.001623780150148094927192721215192250384,
                0.014988684599373741992978104065059596905,
                0.021287595318265635502275046064823982306,
                0.007349500393709578957568417933998716762,
                -0.025127515717112181709014251396183681209,
                -0.052210507359822452833064687638398027048,
                -0.039273839505489904766477593511808663607,
                0.033021568427940004020193498490698402748,
                0.147606943281569008563636202779889572412,
                0.254000252034505602516389899392379447818,
                0.297330876398883392486283128164359368384,
                0.254000252034505602516389899392379447818,
                0.147606943281569008563636202779889572412,
                0.033021568427940004020193498490698402748,
                -0.039273839505489904766477593511808663607,
                -0.052210507359822452833064687638398027048,
                -0.025127515717112181709014251396183681209,
                0.007349500393709578957568417933998716762,
                0.021287595318265635502275046064823982306,
                0.014988684599373741992978104065059596905,
                0.001623780150148094927192721215192250384]
        self.fir = FIR(self.nb_channels, self.fir_coef)
        
    def filter(self, value):
        """
        value: a numpy array of shape (data series, channels)
        """
        for i, x in enumerate(value):  # loop over the data series
            # FIR:
            if self.use_fir:
                x = self.fir.filter(x)
            # notch:
            if self.use_notch:
                denAccum = (x - self.notch_coeff1 * self.dfs[0]) - self.notch_coeff2 * self.dfs[1]
                x = (self.notch_coeff3 * denAccum + self.notch_coeff4 * self.dfs[0]) + self.notch_coeff5 * self.dfs[1]
                self.dfs[1] = self.dfs[0]
                self.dfs[0] = denAccum
            # standardization:
            if self.use_std:
                if self.moving_average is not None:
                    delta = x - self.moving_average
                    self.moving_average = self.moving_average + self.ALPHA_AVG * delta
                    self.moving_variance = (1 - self.ALPHA_STD) * (self.moving_variance + self.ALPHA_STD * delta**2)
                    moving_std = np.sqrt(self.moving_variance)
                    x = (x - self.moving_average) / (moving_std + self.EPSILON)
                else:
                    self.moving_average = x
            try:
                value[i] = x
            except:
                print(f"Error in filtering: {x}")
                continue
        return value

In [20]:
def online_detrend(y, alpha=0.95):
    detrended_y = np.zeros_like(y)
    trend = 0
    for i in range(len(y)):
        trend = alpha * trend + (1 - alpha) * y[i]
        detrended_y[i] = y[i] - trend
    return detrended_y

In [21]:
def raw2filtered(raw):
    '''
    Take in the raw data and filter it online, detrend it
    '''
    filtering4lac = FilterPipeline(nb_channels=1, sampling_rate=250, filter_args=[True, True, False])
    filtered4lac = []
    print(f'Filtering Data Online')
    for i in tqdm(raw):
        filtered4lac.append(filtering4lac.filter(np.array([i])))

    print(f'Detrending Data')
    detrended_data = online_detrend(np.array(filtered4lac).flatten())

    print(f"Running Lacourse")
    data_detect = np.array(detrended_data)
    mask = np.ones(len(data_detect), dtype=bool)
    lacourse = detect_lacourse(
        data_detect,
        mask,
        sampling_rate=250,
    )

    print(f"Lacourse found {len(lacourse)} spindles")

    # Filter data online like on Portiloop
    print(f"Filtering Online with standardization")
    filtering_online = FilterPipeline(nb_channels=1, sampling_rate=250, filter_args=[True, True, True])
    filtered_online = []
    for i in tqdm(raw):
        filtered_online.append(filtering_online.filter(np.array([i])))
    filtered_online = np.array(filtered_online).flatten()

    return detrended_data, lacourse, filtered_online 


In [22]:
path = '/project/portinight-raw/PN_05_EC/'
save_path = '/project/portinight-dataset/'
subject_id = 'PN_05_EC'
age = 24
gender = 'F'

# Iterate through all the csvs in the folder
data = {}
for idx, file in enumerate(os.listdir(path)):
    if file.endswith(".csv"):
        print(f"Processing {file}")

        # Get the filename
        filename = file.split('_')[:-1]
        filename = '_'.join(filename)

        data[filename] = {}

        data[filename]['age'] = age
        data[filename]['gender'] = gender
        df = pd.read_csv(path + file, on_bad_lines='warn', encoding_errors='ignore')
        # convert useful data to floats
        df['converted'] = pd.to_numeric(df.iloc[:, 1], errors='coerce')
        df['converted'] = df['converted'].fillna(method='ffill')
        useful_data = df['converted'].values
        filtered4lac, lacourse, filtered_online = raw2filtered(useful_data)

        spindle_info_mass = {}
        spindle_info_mass[filename] = {
            'onsets': [],
            'offsets': [],
            'labels_num': []
        }

        for spindle in lacourse:
            spindle_info_mass[filename]['onsets'].append(spindle[0])
            spindle_info_mass[filename]['offsets'].append(spindle[1])
            spindle_info_mass[filename]['labels_num'].append(1)

        data[filename]['signal_mass'] = filtered4lac
        data[filename]['signal_filt'] = filtered_online
        data[filename]['ss_label'] = np.ones(len(filtered4lac)) * 5
        data[filename]['spindle_mass_lacourse'] = spindle_info_mass

print(f"Saving f{subject_id}.npz")
np.savez_compressed(os.path.join(save_path, f"{subject_id}.npz"), data)


Processing PN_05_EC_Night4_Sham.csv


Skipping line 2233136: expected 7 fields, saw 10
Skipping line 2233229: expected 7 fields, saw 11

Skipping line 2416571: expected 7 fields, saw 9

Skipping line 2814674: expected 7 fields, saw 8

Skipping line 3565848: expected 7 fields, saw 8

Skipping line 4763812: expected 7 fields, saw 10

Skipping line 5900663: expected 7 fields, saw 8

Skipping line 6283234: expected 7 fields, saw 9

/tmp/ipykernel_819244/312051699.py:21: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + file, on_bad_lines='warn', encoding_errors='ignore')


Filtering Data Online


  0%|          | 0/7506665 [00:00<?, ?it/s]/tmp/ipykernel_819244/1879190733.py:122: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  value[i] = x
 47%|████▋     | 3538416/7506665 [02:08<02:32, 26076.68it/s]

Error in filtering: [97098.71773137]
Error in filtering: [97099.24345356]
Error in filtering: [97100.2568214]
Error in filtering: [97101.87658254]
Error in filtering: [97105.77896172]
Error in filtering: [97111.70820023]
Error in filtering: [97118.92089795]
Error in filtering: [97125.64151395]
Error in filtering: [97130.72280634]
Error in filtering: [97132.68054245]
Error in filtering: [97132.18988964]
Error in filtering: [97132.08260439]
Error in filtering: [97134.16441084]
Error in filtering: [97137.44422506]
Error in filtering: [97139.49732911]
Error in filtering: [97138.4889529]
Error in filtering: [97133.74313537]
Error in filtering: [97126.05583142]
Error in filtering: [97116.72648758]
Error in filtering: [97107.07442473]
Error in filtering: [97098.36236807]
Error in filtering: [97091.48731642]
Error in filtering: [97086.57657143]
Error in filtering: [97083.22072218]
Error in filtering: [97081.50575322]
Error in filtering: [97081.88759037]
Error in filtering: [97084.99140042]
Err

...

 47%|████▋     | 3565594/7506665 [02:15<14:55, 4401.08it/s]

Error in filtering: [98170.92438799]
Error in filtering: [98172.00081666]
Error in filtering: [98172.18106455]
Error in filtering: [98171.88300892]
Error in filtering: [98172.31322968]
Error in filtering: [98173.18080617]
Error in filtering: [98172.55345078]
Error in filtering: [98169.00199359]
Error in filtering: [98161.2717149]
Error in filtering: [98150.95106639]
Error in filtering: [98140.53398377]
Error in filtering: [98132.38181311]
Error in filtering: [98126.92561967]
Error in filtering: [98122.02591399]
Error in filtering: [98116.19439297]
Error in filtering: [98107.13244767]
Error in filtering: [98095.00788557]
Error in filtering: [98081.51585894]
Error in filtering: [98070.49984691]
Error in filtering: [98065.0210907]
Error in filtering: [98066.1457128]
Error in filtering: [98072.37761863]
Error in filtering: [98081.46063243]
Error in filtering: [98090.4010061]
Error in filtering: [98096.82811481]
Error in filtering: [98100.10946424]
Error in filtering: [98102.3365421]
Error 

 48%|████▊     | 3565819/7506665 [02:15<02:29, 26326.30it/s]


Error in filtering: [97893.60884104]
Error in filtering: [97897.10481779]
Error in filtering: [97895.48344098]
Error in filtering: [97893.47268807]
Error in filtering: [97897.21926986]
Error in filtering: [97910.57281657]
Error in filtering: [97931.04023572]
Error in filtering: [97952.36215232]
Error in filtering: [97967.71320043]
Error in filtering: [97974.7456388]
Error in filtering: [97975.27872499]
Error in filtering: [97972.08769604]
Error in filtering: [97969.57300023]
Error in filtering: [97967.92191215]
Error in filtering: [97968.26019861]
Error in filtering: [97971.35473994]
Error in filtering: [97977.30595384]
Error in filtering: [97985.93103937]
Error in filtering: [97994.73813535]
Error in filtering: [98000.23595431]
Error in filtering: [97999.49309134]
Error in filtering: [97991.79032043]
Error in filtering: [97980.58228816]
Error in filtering: [97971.28150609]
Error in filtering: [97970.17756569]
Error in filtering: [97978.12007622]
Error in filtering: [97991.61833365]
Er

ValueError: could not convert string to float: '9.583804408765365952e+01885838e+05'

In [36]:
# Count how many NaNs are in the data
df['converted'].isna().sum()

0

In [26]:
save_path = '/project/portinight-dataset/'
subject_id = 'PN_01_HJ'

# Try loading the portinight data:
data = np.load(os.path.join(save_path, f"{subject_id}.npz"), allow_pickle=True)

In [27]:

data['arr_0']

array({'PN_01_HJ_Night1': {'age': 28, 'gender': 'M', 'signal_mass': array([  16.36395368,  248.29117582, 1204.59631932, ...,   62.28175158,
         78.37960546,   90.34512397]), 'signal_filt': array([17.2252144 , 28.4747362 , 27.93541527, ...,  1.04844287,
        1.33055178,  1.51649733]), 'ss_label': array([5., 5., 5., ..., 5., 5., 5.]), 'spindle_mass_lacourse': {'PN_01_HJ_Night1': {'onsets': [4375, 13300, 14450, 24650, 32450, 34425, 38500, 39400, 43400, 56875, 63725, 65950, 66650, 71600, 79900, 87250, 98825, 107025, 114450, 129324, 130250, 140375, 146475, 150000, 156450, 169300, 169775, 172425, 178200, 181675, 188825, 189100, 190150, 190400, 190725, 192075, 195850, 202325, 207225, 207925, 209025, 209900, 213200, 214075, 222250, 222675, 224950, 227100, 228000, 229325, 239200, 240275, 241500, 242475, 249575, 250550, 254850, 256925, 263325, 263775, 264700, 267775, 271150, 273125, 279225, 281900, 296650, 300350, 309450, 342000, 343650, 347350, 347875, 355700, 370750, 392475, 404150, 41

In [60]:
data.keys()

dict_keys(['PN_01_HJ_Night1', 'PN_01_HJ_Night4', 'PN_01_HJ_Night5', 'PN_01_HJ_Night6', 'PN_01_HJ_Night3'])

In [66]:
# clean the dictionary keys
for new_key in list(data.keys()):
    print(data[new_key]['spindle_mass_lacourse'].keys())
    old_key = list(data[new_key]['spindle_mass_lacourse'].keys())[0]
    print(old_key)
    data[new_key]['spindle_mass_lacourse'][new_key] = data[new_key]['spindle_mass_lacourse'].pop(old_key)

dict_keys(['PN_01_HJ_Night1_Stim_night0'])
PN_01_HJ_Night1_Stim_night0
dict_keys(['PN_01_HJ_Night4_Sham_night1'])
PN_01_HJ_Night4_Sham_night1
dict_keys(['PN_01_HJ_Night5_Stim_night2'])
PN_01_HJ_Night5_Stim_night2
dict_keys(['PN_01_HJ_Night6_Sham_night5'])
PN_01_HJ_Night6_Sham_night5
dict_keys(['PN_01_HJ_Night3_Stim_night6'])
PN_01_HJ_Night3_Stim_night6


In [67]:
data['PN_01_HJ_Night1']['spindle_mass_lacourse'].keys()

dict_keys(['PN_01_HJ_Night1'])

In [69]:
# Save the cleaned data
np.savez_compressed(os.path.join(save_path, f"{subject_id}.npz"), data)

In [13]:
from portiloopml.portiloop_python.ANN.data.mass_data_new import MassDataset

data_path = '/project/MASS/mass_spindles_dataset/'
subjects = ['PN_01_HJ_Night4', '01-01-0001']

dataset = MassDataset(data_path, 30, 30, 30, subjects=subjects, use_filtered=False, sampleable='spindles', compute_spindle_labels=False)

Time taken to load PN_01_HJ_Night4: 21.723674297332764
Time taken to load 01-01-0001: 2.1120471954345703
Time taken to create lookup table: 1.442598819732666
Number of sampleable indices: 14841701
Number of spindle indexes: 422714
Number of spindles: 2301
Number of N1 indexes: 0
Number of N2 indexes: 0
Number of N3 indexes: 0
Number of R indexes: 0
Number of W indexes: 0


In [16]:
# Read the list of subjects from the file
subject_list_path = '/home/ubuntu/portiloop-training/subjects_portinight.txt'

with open(subject_list_path, 'r') as f:
    subjects = f.readlines()

subjects = [x.strip() for x in subjects]


for subject in subjects:
    dataset = MassDataset(data_path, 30, 30, 30, subjects=[subject], use_filtered=False, sampleable='spindles', compute_spindle_labels=False)

Time taken to load PN_01_HJ_Night1: 3.674607992172241
Time taken to create lookup table: 0.25005435943603516
Number of sampleable indices: 3762866
Number of spindle indexes: 78774
Number of spindles: 661
Number of N1 indexes: 0
Number of N2 indexes: 0
Number of N3 indexes: 0
Number of R indexes: 0
Number of W indexes: 0
Time taken to load PN_01_HJ_Night5: 3.527395248413086
Time taken to create lookup table: 0.688335657119751
Number of sampleable indices: 7961032
Number of spindle indexes: 0
Number of spindles: 0
Number of N1 indexes: 0
Number of N2 indexes: 0
Number of N3 indexes: 0
Number of R indexes: 0
Number of W indexes: 0
Time taken to load PN_01_HJ_Night3: 3.519167423248291
Time taken to create lookup table: 0.4974982738494873
Number of sampleable indices: 6372760
Number of spindle indexes: 124047
Number of spindles: 1035
Number of N1 indexes: 0
Number of N2 indexes: 0
Number of N3 indexes: 0
Number of R indexes: 0
Number of W indexes: 0
Time taken to load PN_01_HJ_Night4: 3.566